# Lambda School Data Science - A First Look at Data



## Lecture - let's explore Python DS libraries and examples!

The Python Data Science ecosystem is huge. You've seen some of the big pieces - pandas, scikit-learn, matplotlib. What parts do you want to see more of?

In [0]:
# TODO - we'll be doing this live, taking requests
# and reproducing what it is to look up and learn things

myDict = {'what': 1, 'a': 2, 'silly': 3, 'dictionary': 4}

This is some text

## Assignment - now it's your turn

Pick at least one Python DS library, and using documentation/examples reproduce in this notebook something cool. It's OK if you don't fully understand it or get it 100% working, but do put in effort and look things up.

In [0]:
# TODO - your code here
# Use what we did live in lecture as an example
import pandas as pd

In [47]:
# Read in CSV file of heart disease data in Cleveland
df = pd.read_csv('https://archive.ics.uci.edu/ml/machine-learning-databases/heart-disease/processed.cleveland.data')
# Show first 5 rows of data
df.head()

,63.0,1.0,1.0.1,145.0,233.0,1.0.2,2.0,150.0,0.0,2.3,3.0,0.0.1,6.0,0
0,67.0,1.0,4.0,160.0,286.0,0.0,2.0,108.0,1.0,1.5,2.0,3.0,3.0,2
1,67.0,1.0,4.0,120.0,229.0,0.0,2.0,129.0,1.0,2.6,2.0,2.0,7.0,1
2,37.0,1.0,3.0,130.0,250.0,0.0,0.0,187.0,0.0,3.5,3.0,0.0,3.0,0
3,41.0,0.0,2.0,130.0,204.0,0.0,2.0,172.0,0.0,1.4,1.0,0.0,3.0,0
4,56.0,1.0,2.0,120.0,236.0,0.0,0.0,178.0,0.0,0.8,1.0,0.0,3.0,0


In [0]:
# Rename some columns to make them identifiable
df.rename(columns={'63.0': 'Age', '1.0':'Sex', '150.0':'thalach', '0.0.1': 'ca',
                    '6.0': 'thal', '2.3': 'oldpeak', '0': 'num'}, inplace=True)

In [0]:
# Drop the rest of the columns
df.drop(['1.0.1', '145.0', '233.0', '1.0.2', '2.0', '0.0', '3.0'], axis=1, inplace=True)

In [50]:
# Check to make sure our changes worked as planned
df.head()

,Age,Sex,thalach,oldpeak,ca,thal,num
0,67.0,1.0,108.0,1.5,3.0,3.0,2
1,67.0,1.0,129.0,2.6,2.0,7.0,1
2,37.0,1.0,187.0,3.5,0.0,3.0,0
3,41.0,0.0,172.0,1.4,0.0,3.0,0
4,56.0,1.0,178.0,0.8,0.0,3.0,0


In [51]:
# Check out the shape of dataframe - see how many datapoints we have
df.shape

(302, 7)

In [52]:
# Look at the basic analyses of the data - 2 of the columns got dropped from this analysis
df.describe()

,Age,Sex,thalach,oldpeak,num
count,302.000000,302.000000,302.000000,302.000000,302.000000
mean,54.410596,0.678808,149.605960,1.035430,0.940397
std,9.040163,0.467709,22.912959,1.160723,1.229384
min,29.000000,0.000000,71.000000,0.000000,0.000000
25%,48.000000,0.000000,133.250000,0.000000,0.000000
50%,55.500000,1.000000,153.000000,0.800000,0.000000
75%,61.000000,1.000000,166.000000,1.600000,2.000000
max,77.000000,1.000000,202.000000,6.200000,4.000000


In [53]:
# Looking at the data types we see the two dropped columns have object dtypes
df.dtypes

Age        float64
Sex        float64
thalach    float64
oldpeak    float64
ca          object
thal        object
num          int64
dtype: object

I tried to drop the NaN values in 'ca' and 'thal' but the missing values are input as '?' and dont show up as NaN

### Assignment questions

After you've worked on some code, answer the following questions in this text block:

1.  Describe in a paragraph of text what you did and why, as if you were writing an email to somebody interested but nontechnical.

2.  What was the most challenging part of what you did?

3.  What was the most interesting thing you learned?

4.  What area would you like to explore with more time?




# 1. I uploaded and explored a dataset of patients who were looked at for Heart Disease in Cleveland. The data has many different values from tests run on the patients, and one column indicating, on a scale from 0-4, if they have heart disease. I renamed the columns to make them identifiable as to what they measured, and I dropped a bunch of the columns too. I also looked at some basic measures of the data in each column,  like max, mean, std-dev.

# 2. Getting the data into the DataFrame was a bit tricky. I ended up using a dataset I had found the raw data for and looked at previously. There seem to be multiple ways of getting the data in, but for each you need to know the right file and file path to call.

# 3. The most interesting thing I learned was how to upload files from my computer directly into colab, although that was after I had started on the data that I did.

# 4. With more time, I would figure out how to remove the '?' from the data. I would also want to get more comfortable going from empty notebook to reading in data, however was necessary given the data file.

## Stretch goals and resources

Following are *optional* things for you to take a look at. Focus on the above assignment first, and make sure to commit and push your changes to GitHub (and since this is the first assignment of the sprint, open a PR as well).

- [pandas documentation](https://pandas.pydata.org/pandas-docs/stable/)
- [scikit-learn documentation](http://scikit-learn.org/stable/documentation.html)
- [matplotlib documentation](https://matplotlib.org/contents.html)
- [Awesome Data Science](https://github.com/bulutyazilim/awesome-datascience) - a list of many types of DS resources

Stretch goals:

- Find and read blogs, walkthroughs, and other examples of people working through cool things with data science - and share with your classmates!
- Write a blog post (Medium is a popular place to publish) introducing yourself as somebody learning data science, and talking about what you've learned already and what you're excited to learn more about.